In [1]:
%load_ext autoreload
%autoreload 2
import sys
import keras
from keras.models import Model
from keras.layers import Conv2D, UpSampling2D
sys.path.insert(0, "../scripts/")
sys.path.insert(0, "../china/Deep-Image-Matting")
from load_encoder_decoder import build_encoder_decoder_from_vgg
# from data_generator import DataGenerator
from losses import overall_loss_wrapper
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import multi_gpu_model
from keras.models import load_model
from keras.layers import Input
import numpy as np
from unpooling import Unpooling
from matplotlib.pyplot import imshow, figure
from model import build_encoder_decoder, build_refinement
import cv2 as cv
import math
import tensorflow as tf
from scipy import misc

Using TensorFlow backend.


In [2]:
model = load_model("C:\\Workspace\\ThesisImageMatting\\model_checkpoints\\encoder_decoder.07-val_loss-0.1294-val_sad-187.0877-val_mse-0.0446.hdf5", custom_objects={"Unpooling": Unpooling()})

C:\Users\thalv\Anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [23]:
model_new = build_encoder_decoder_from_vgg()

ValueError: Tried to convert 'shape' to a tensor and failed. Error: None values not supported.

In [4]:
model.layers.pop(0)

In [5]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
block1_conv1 (Conv2D)           (None, 320, 320, 64) 2368        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 320, 320, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 160, 160, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_conv1 (Conv2D)           (None, 160, 160, 128 73856       block1_pool[0][0]                
__________________________________________________________________________________________________
block2_con

In [7]:
newInput = Input(shape=(None, None, 4))    # let us say this new InputLayer


In [8]:
out = model(newInput)

In [11]:
newModel = Model(newInput, out)

In [12]:
newModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 4)     0         
_________________________________________________________________
model_1 (Model)              (None, 320, 320, 1)       130538945 
Total params: 130,538,945
Trainable params: 130,537,921
Non-trainable params: 1,024
_________________________________________________________________


In [13]:
img2 = cv.imread("D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Test_set\\test_ds\\sieve-641426_1920_7.jpg")
trimap_2 = cv.imread("D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Test_set\\Adobe-licensed images\\trimaps\\sieve-641426_1920_7.png")
gt_2 = cv.imread("D:\\Image Matting Dataset\\adobe dataset\\Combined_Dataset\\Test_set\\Adobe-licensed images\\alpha\\sieve-641426_1920.png")

In [14]:
img2.shape

(1441, 1920, 3)

In [15]:
input_for_nn = np.empty((1441, 1920, 4))

input_for_nn[:, :, 0:3] = img2
input_for_nn[:, :, 3] = trimap_2[:, :, 0]

In [17]:
model.predict(input_for_nn.reshape(1, 1441,1920, 4))

ValueError: Error when checking input: expected input_2 to have shape (320, 320, 4) but got array with shape (1441, 1920, 4)

In [21]:
for i in range(len(model.layers)):
    print(model.layers[i].weights)

[<tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 4, 64) dtype=float32_ref>, <tf.Variable 'block1_conv1/bias:0' shape=(64,) dtype=float32_ref>]
[<tf.Variable 'block1_conv2/kernel:0' shape=(3, 3, 64, 64) dtype=float32_ref>, <tf.Variable 'block1_conv2/bias:0' shape=(64,) dtype=float32_ref>]
[]
[<tf.Variable 'block2_conv1/kernel:0' shape=(3, 3, 64, 128) dtype=float32_ref>, <tf.Variable 'block2_conv1/bias:0' shape=(128,) dtype=float32_ref>]
[<tf.Variable 'block2_conv2/kernel:0' shape=(3, 3, 128, 128) dtype=float32_ref>, <tf.Variable 'block2_conv2/bias:0' shape=(128,) dtype=float32_ref>]
[]
[<tf.Variable 'block3_conv1/kernel:0' shape=(3, 3, 128, 256) dtype=float32_ref>, <tf.Variable 'block3_conv1/bias:0' shape=(256,) dtype=float32_ref>]
[<tf.Variable 'block3_conv2/kernel:0' shape=(3, 3, 256, 256) dtype=float32_ref>, <tf.Variable 'block3_conv2/bias:0' shape=(256,) dtype=float32_ref>]
[<tf.Variable 'block3_conv3/kernel:0' shape=(3, 3, 256, 256) dtype=float32_ref>, <tf.Variable 'block3_conv3/